In [118]:
import pandas as pd
import numpy as np
import shutil
import os
import random
from PIL import Image
from datasets import load_dataset

In [119]:
plots = pd.read_json("../data/annotations.json")

print("Plots Dataset size :", len(plots))
plots.head()

Plots Dataset size : 33657


,models,type,general_figure_info,image_index
0,"[{'name': 'Goods and services', 'color': '#F0E...",dot_line,{'title': {'text': 'Payments made towards prim...,0
1,"[{'name': 'Arms exports', 'color': '#FFFF00', ...",dot_line,{'title': {'text': 'Arm exports and imports in...,1
2,"[{'name': 'Infant', 'color': '#191970', 'label...",dot_line,{'title': {'text': 'Mortality rates in Marshal...,2
3,"[{'name': 'Female', 'color': '#D2B48C', 'label...",dot_line,{'title': {'text': 'Percentage of repeaters in...,3
4,"[{'name': 'US$', 'color': '#B8860B', 'label': ...",dot_line,{'title': {'text': 'Total resource balance of ...,4


In [120]:
plots["type"].value_counts()

type
hbar_categorical    11292
vbar_categorical    11242
dot_line             5574
line                 5549
Name: count, dtype: int64

**Filtro el dataset para quedarme con las visualizaciones single-plot, elimino los hbar plots y renombro los tipos**

In [121]:
single_plots = plots[plots['models'].apply(lambda x: len(x) == 1)].copy()

#single_plots.loc[:, "type"] = single_plots["type"].map({"dot_line": "circle", "vbar_categorical": "bar", "line": "line", "hbar_categorical": "hbar"})
#single_plots.loc[:, "type"] = single_plots["type"].map({"dot_line": "circle", "vbar_categorical": "bar", "line": "line"})

single_plots = single_plots.dropna(subset=["type"])

print("Single-plots Dataset size :", len(single_plots))
single_plots.head()

Single-plots Dataset size : 6070


,models,type,general_figure_info,image_index
4435,"[{'name': 'Secure Internet servers', 'color': ...",dot_line,{'title': {'text': 'Number of secure servers i...,4435
4436,"[{'name': 'Communications', 'color': '#FFA07A'...",dot_line,"{'y_axis': {'minor_ticks': {'values': [0, 1, 2...",4436
4437,"[{'name': 'Armed forces personnel', 'color': '...",dot_line,{'title': {'text': 'Total onduty military pers...,4437
4438,[{'name': 'Government expenditure per tertiary...,dot_line,{'title': {'text': 'Government expenditure per...,4438
4439,"[{'name': 'Tax payments', 'color': '#6B8E23', ...",dot_line,"{'y_axis': {'minor_ticks': {'values': [0, 1, 2...",4439


In [ ]:
def check_for_quotes():

    x_values_new = []
    
    for x_val in x_values:
        if isinstance(x_val, str):
            if x_val.find("'") != -1: x_val = x_val.replace("'","")

        x_values_new.append(x_val)

    return x_values_new

In [ ]:
new_df_data = []

for idx, row in single_plots.iterrows():

    mark_type, x_type, y_type, x_name, y_name, x_values, y_values = None, None, None, None, None, None, None

    x_name = row["general_figure_info"]["x_axis"]["label"]["text"]
    y_name = row["general_figure_info"]["y_axis"]["label"]["text"]

    x_values = row["models"][0]["x"]
    y_values = row["models"][0]["y"]

    # HACER UNA FUNCION AUX <-------------------------------------------- Y HACER ESTO PARA AMBOS VALORES



    # <-----------------------------------------------------------------

    if row["type"] == "dot_line":
        
        mark_type = "circle"
        
        if x_name == "Year":
            x_type = "temporal" 
            x_values = row["general_figure_info"]["x_axis"]["major_labels"]["values"]
        else:
            x_type = "quantitative"

        y_type = "quantitative"

    elif row["type"] == "vbar_categorical":
        
        mark_type = "bar"

        x_type = var_type(x_values[0])
        y_type = "quantitative"

    elif row["type"] == "hbar_categorical":

        mark_type = "bar"

        x_type = "quantitative"
        y_type = var_type(y_values[0])
        
    elif row["type"] == "line":
        
        mark_type = "line"

        x_type = "temporal"
        y_type = "quantitative"

    new_df_data.append({"mark_type": mark_type,
                        "x_type": x_type,
                        "y_type": y_type,
                        "x_name": x_name,
                        "y_name": y_name,
                        "x_values": x_values,
                        "y_values": y_values})

    print(idx, new_df_data[-1])

4435 {'mark_type': 'circle', 'x_type': 'temporal', 'y_type': 'quantitative', 'x_name': 'Year', 'y_name': 'Number of servers(per 1 million people)', 'x_values': [0, 1, 2, 3, 4, 5], 'y_values': [0.260907839263029, 0.5062283808842091, 1.22968619883829, 1.1969067142876, 1.39984527043611, 1.13782884960497]}
4436 {'mark_type': 'circle', 'x_type': 'temporal', 'y_type': 'quantitative', 'x_name': 'Year', 'y_name': 'Cost of communications, computer etc. (% of service exports)', 'x_values': [0, 1, 2, 3], 'y_values': [16.9288755558139, 16.0622961382481, 18.283211989346, 15.0966569292663]}
4437 {'mark_type': 'circle', 'x_type': 'temporal', 'y_type': 'quantitative', 'x_name': 'Year', 'y_name': 'Armed forces personnel', 'x_values': [0, 1, 2, 3, 4, 5, 6], 'y_values': [5000, 5000, 5000, 4000, 5000, 6000, 7000]}
4438 {'mark_type': 'circle', 'x_type': 'temporal', 'y_type': 'quantitative', 'x_name': 'Year', 'y_name': 'Expenditure (as % of GDP per capita)', 'x_values': [0, 1, 2, 3], 'y_values': [57.7957916

In [152]:
single_plots["models"][4435]

[{'name': 'Secure Internet servers',
  'color': '#D2B48C',
  'label': 'Secure Internet servers',
  'bboxes': [{'y': 481, 'x': 84, 'w': 13, 'h': 13},
   {'y': 387, 'x': 239, 'w': 13, 'h': 13},
   {'y': 111, 'x': 393, 'w': 13, 'h': 13},
   {'y': 123, 'x': 548, 'w': 13, 'h': 13},
   {'y': 46, 'x': 702, 'w': 13, 'h': 13},
   {'y': 146, 'x': 857, 'w': 13, 'h': 13}],
  'y': [0.260907839263029,
   0.5062283808842091,
   1.22968619883829,
   1.1969067142876,
   1.39984527043611,
   1.13782884960497],
  'x': [0, 1, 2, 3, 4, 5]}]

In [173]:
single_plots["general_figure_info"][4435]["x_axis"]["major_labels"]["values"]

['2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013']

In [153]:
it = iter(single_plots[single_plots["type"] == "vbar_categorical"]["models"])

In [154]:
next(it)

[{'name': 'GHG net emissions or removals',
  'labels': 'GHG net emissions or removals',
  'width': 0.15,
  'colors': '#9ACD32',
  'bboxes': [{'y': 57, 'x': 84, 'w': 34, 'h': 530},
   {'y': 778, 'x': 308, 'w': 34, 'h': -191},
   {'y': 1403, 'x': 531, 'w': 34, 'h': -816},
   {'y': 1906, 'x': 755, 'w': 34, 'h': -1319}],
  'y': [74.4790781926427,
   -26.8583882556454,
   -114.609578811852,
   -185.326720711572],
  'x': ['1991', '1992', '1993', '1994']}]

In [161]:
def var_type(var):

    try:
        float(var)
        return "temporal"
    except:
        return "nominal"

def get_vega(df, index, vega_structure = True):
    
    ''' Get vega code from a PlotQA DataFrame entry at index '''

    mark_type = df["type"][index]

    x_name = df["general_figure_info"][index]["x_axis"]["label"]["text"]
    y_name = df["general_figure_info"][index]["y_axis"]["label"]["text"]

    y_values = [np.round(y, 2) for y in df["models"][index][0]["y"]]
    
    x_values = single_plots["general_figure_info"][index]["x_axis"]["major_labels"]["values"]
    x_values_new = []
    
    for x_val in x_values:
        if isinstance(x_val, str):
            if x_val.find("'") != -1: x_val = x_val.replace("'","")

        x_values_new.append(x_val)

    x_values = x_values_new    

    midpoint = len(x_values) // 2
    x_values = x_values[:midpoint]

    x_type = var_type(x_values[0])
    y_type = var_type(y_values[0])

    vega = None

    if vega_structure:

        vega = {"mark": mark_type,
                "encoding": {
                    "x": {"field": x_name, "type": x_type},
                    "y": {"field": y_name, "type": "quantitative"}
                },
                "data": {
                    "values": [
                        {"x": x, "y": y} for x,y in zip(x_values, y_values)
                    ]
                }
            }
    else:
        vega = mark_type + "|" + x_type + "|" + x_name + "|" + y_type + "|" + y_name

    return vega

In [ ]:
index = single_plots.sample().index[0]
get_vega(single_plots, index)

In [ ]:
Image.open("../plotqa/png/"+ str(index) +".png")

In [ ]:
def erase_files(path):

    for filename in os.listdir(path):

        file_path = os.path.join(path, filename)

        try:
            if os.path.isfile(file_path): os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

def copy_file(filename, src_dir, dest_dir):
    
    source_path = os.path.join(src_dir, filename)
    destination_path = os.path.join(dest_dir,filename)

    if os.path.exists(source_path):
        try:
            shutil.copy(source_path, destination_path)
        except IOError as e:
            print(f"Failed to copy {filename}: {e}")
    else:
        print(f"File {filename} does not exist in the source directory.")

def generate_split(path, df, indexes):
    
    print("generating slice of size ", len(indexes))
    erase_files(path)

    metadata = [['file_name','text']]

    for i in indexes:
        
        copy_file(str(i) + ".png", "../plotqa/png", path)

        vega_str = '"' + str(get_vega(df, i, vega_structure = False)) + '"'
        metadata.append([str(i) + ".png", vega_str])

    np.savetxt(path + "/metadata.csv", metadata, delimiter = ',', fmt = '% s')

def generate_dataset(path, df, n_samples):
    
    np.random.seed(42)
    random.seed(42)

    n_train = n_samples
    n_val = int(n_samples * 0.02)
    n_test = int(n_samples * 0.1)

    indexes = list(df.sample(n_train + n_val + n_test).index)

    generate_split(path + "/train", df, indexes[:n_train])
    generate_split(path + "/val", df, indexes[n_train:n_train+n_val])
    generate_split(path + "/test", df, indexes[n_train+n_val:])

**Dataset Generation**

In [ ]:
dataset_name = "plotqa_simple_2"
generate_dataset("datasets/" + dataset_name, single_plots, n_samples = 1000)

**Upload to Huggingface**

In [ ]:
dataset = load_dataset("imagefolder", data_dir = "datasets/" + dataset_name)
dataset.push_to_hub("martinsinnona/" + dataset_name)

In [ ]:
x_types = {"quantitative":0, "temporal":0, "nominal":0, "ordinal":0}
y_types = {"quantitative":0, "temporal":0, "nominal":0, "ordinal":0}

for i in single_plots.index:

    vega = get_vega(single_plots, i)

    x_types[ vega["encoding"]["x"]["type"] ] += 1
    y_types[ vega["encoding"]["y"]["type"] ] += 1

print(x_types)
print(y_types)